# Exercise 11.1 
Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [12]:
import first
import statsmodels.formula.api as smf

In [20]:
live, firsts, others = first.MakeFrames()
# Getting pregnancy data greater than the 30th week of pregnancy.
live = live[live.prglngth>30] 

# I believe the following are the variables have a statistically significant effect on pregnancy length.
model = smf.ols('prglngth ~ birthord==1 + race + nbrnaliv>1', data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     31.27
Date:                Fri, 12 Nov 2021   Prob (F-statistic):           4.24e-20
Time:                        15:32:51   Log-Likelihood:                -18252.
No. Observations:                8884   AIC:                         3.651e+04
Df Residuals:                    8880   BIC:                         3.654e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7581      0.070    553.685      0.000      38.621      38.895
birthord == 1[T.True]     0.1054      0.040      2.625      0.009       0.027       0.184
nbrnaliv > 1[T.True]     -1.4876      0.165     -9.038      0.000      -1.810      -1.165
race                      0.0502      0.035      1.420      0.156      -0.019       0.120
==============================================================================
Omnibus:                     1574.506   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6122.061
Skew:                          -0.844   Prob(JB):                         0.00
Kurtosis:                       6.700   Cond. No.                         18.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Exercise 11.3 
If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called poisson. It works the same way as ols and logit. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called numbabes.
Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [55]:
import nsfg
import numpy as np
import pandas as pd

In [56]:
resp = nsfg.ReadFemResp()

In [57]:
join = live.join(resp, on='caseid', rsuffix='_r')
formula='numbabes ~ age_r + C(race) + totincr + educat'

#creating a model below using Poisson regression
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.393384
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 5651
Model:                        Poisson   Df Residuals:                     5645
Method:                           MLE   Df Model:                            5
Date:                Fri, 12 Nov 2021   Pseudo R-squ.:                  0.1073
Time:                        16:10:33   Log-Likelihood:                -7874.0
converged:                       True   LL-Null:                       -8820.6
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.2055      0.082    -14.686      0.000      -1.366      -1.045
C(race)[T.2]    -0.0256      0.028     -0.921      0.357      -0.080       0.029
C(race)[T.3]    -0.1977      0.050     -3.927      0.000      -0.296      -0.099
age_r            0.0616      0.002     40.209      0.000       0.059       0.065
totincr         -0.0520      0.003    -16.955      0.000      -0.058      -0.046
educat          -0.0067      0.005     -1.406      0.160      -0.016       0.003
================================================================================
"""

In [58]:
# Predict the number of children for a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000
columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35, 1, 14, 16]], columns=columns)
results.predict(new)

0    1.124008
dtype: float64

# Exercise 11.4 
If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called mnlogit. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called rmarital.
Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [61]:
# Creating the model using multinomial logistic regression.

formula='rmarital ~ age_r + C(race) + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.188934
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 5651
Model:                        MNLogit   Df Residuals:                     5621
Method:                           MLE   Df Model:                           25
Date:                Fri, 12 Nov 2021   Pseudo R-squ.:                 0.08755
Time:                        16:17:49   Log-Likelihood:                -6718.7
converged:                       True   LL-Null:                       -7363.3
Covariance Type:            nonrobust   LLR p-value:                1.647e-256
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.5003      0.324      7.709      0.000       1.865       3.136
C(race)[T.2]    -1.1476      0.106    -10.829      0.000      -1.355      -0.940
C(race)[T.3]    -0.7534      0.170     -4.426      0.000      -1.087      -0.420
age_r           -0.0075      0.005     -1.384      0.166      -0.018       0.003
totincr          0.0347      0.012      2.889      0.004       0.011       0.058
educat          -0.2789      0.021    -13.298      0.000      -0.320      -0.238
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.9400      0.955     -0.984      0.325      -2.813       0.933
C(race)[T.2]    -0.8189      0.313     -2.619      0.009      -1.432      -0.206
C(race)[T.3]    -1.1772      0.638     -1.844      0.065      -2.428       0.074
age_r            0.0113      0.016      0.689      0.491      -0.021       0.044
totincr         -0.0090      0.035     -0.255      0.799      -0.078       0.060
educat          -0.2202      0.061     -3.615      0.000      -0.340      -0.101
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.8573      0.310     -2.764      0.006      -1.465      -0.249
C(race)[T.2]    -0.5505      0.115     -4.771      0.000      -0.777      -0.324
C(race)[T.3]    -0.7380      0.203     -3.644      0.000      -1.135      -0.341
age_r            0.0005      0.005      0.092      0.927      -0.010       0.011
totincr          0.0085      0.012      0.715      0.475      -0.015       0.032
educat          -0.0396      0.018     -2.177      0.029      -0.075      -0.004
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0815      0.389      0.210      0.834      -0.680       0.843
C(race)[T.2]    -1.3716      0.127    -10.817      0.000      -1.620      -1.123
C(race)[T.3]    -0.4130      0.182     -2.274      0.023      -0.769      -0.057
age_r           -0.0040      0.007     -0.602      0.547      -0.017       0.009
totincr          0.0926      0.015      5.980      0.000       0.062       0.123
educat          -0.1634      0.024     -6.802      0.000      -0.210      -0.116
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.2030      0.301     13.967      0.000       3.613       4.793
C

In [62]:
# Predicting for a woman who is 25 years old, white and a high school graduate whose annual household income is about $45,000.

# This person has a 63% chance of being currently married a 10% chance of being "not married but living with opposite sex partner", etc.

columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 2, 11, 12]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.632409,0.104414,0.009325,0.107,0.061406,0.085446
